In [ ]:
!git clone https://github.com/sequence-toolbox/SeQUeNCe.git


Cloning into 'SeQUeNCe'...
remote: Enumerating objects: 13048, done.
remote: Counting objects: 100% (3893/3893), done.
remote: Compressing objects: 100% (1155/1155), done.
remote: Total 13048 (delta 2881), reused 3564 (delta 2699), pack-reused 9155 (from 1)
Receiving objects: 100% (13048/13048), 20.04 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (9279/9279), done.


In [ ]:
cd /content/SeQUeNCe/example


/content/SeQUeNCe/example


In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.qkd.BB84 import pair_bb84_protocols
from sequence.topology.router_net_topo import RouterNetTopo

In [ ]:
!pip install sequence

In [ ]:
class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []

    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) # interface for BB84 to generate key

    def pop(self, info): # interface for BB84 to return generated keys
        self.keys.append(info)
        self.times.append(self.timeline.now() * 1e-9)

In [ ]:
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    # begin by defining the simulation timeline with the correct simulation time
    tl = Timeline(sim_time * 1e9)

    # Here, we create nodes for the network (QKD nodes for key distribution)
    # stack_size=1 indicates that only the BB84 protocol should be included
    n1 = QKDNode("n1", tl, stack_size=1)
    n2 = QKDNode("n2", tl, stack_size=1)
    n1.set_seed(0)
    n2.set_seed(1)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])

    # connect the nodes and set parameters for the fibers
    # note that channels are one-way
    # construct a classical communication channel
    # (with arguments for the channel name, timeline, and length (in m))
    cc0 = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc1 = ClassicalChannel("cc_n2_n1", tl, distance=1e3)
    cc0.set_ends(n1, n2.name)
    cc1.set_ends(n2, n1.name)
    # construct a quantum communication channel
    # (with arguments for the channel name, timeline, attenuation (in db/m), and distance (in m))
    qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc0.set_ends(n1, n2.name)
    qc1.set_ends(n2, n1.name)

    # instantiate our written keysize protocol
    km1 = KeyManager(tl, keysize, 25)
    km1.lower_protocols.append(n1.protocol_stack[0])
    n1.protocol_stack[0].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 25)
    km2.lower_protocols.append(n2.protocol_stack[0])
    n2.protocol_stack[0].upper_protocols.append(km2)

    # start simulation and record timing
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))

    # display our collected metrics
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ms)")
    plt.ylabel("Number of Completed Keys")
    plt.show()

    print("key error rates:")
    for i, e in enumerate(n1.protocol_stack[0].error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

In [ ]:
interactive_plot = interact(test, sim_time=(0, 1, 0.1), keysize=[128, 256, 512])
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='sim_time', max=1.0), Dropdown(description='keysize',…

<function __main__.test(sim_time, keysize)>